In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
data=pd.read_csv("spam.csv",encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
data=data.drop(labels=["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis=1)
data.rename(columns={"v1":"label","v2":"Message"},inplace=True)
data.head(2)

,label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [4]:
data=pd.get_dummies(data=data,columns=["label"],dtype='int')
data.head(5)

,Message,label_ham,label_spam
0,"Go until jurong point, crazy.. Available only ...",1,0
1,Ok lar... Joking wif u oni...,1,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,0,1
3,U dun say so early hor... U c already then say...,1,0
4,"Nah I don't think he goes to usf, he lives aro...",1,0


In [5]:

data['label_ham'].value_counts()

label_ham
1    4825
0     749
Name: count, dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5574 entries, 0 to 5573
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Message     5574 non-null   object
 1   label_ham   5574 non-null   int32 
 2   label_spam  5574 non-null   int32 
dtypes: int32(2), object(1)
memory usage: 87.2+ KB


In [7]:
data.describe()

,label_ham,label_spam
count,5574.000000,5574.000000
mean,0.865626,0.134374
std,0.341084,0.341084
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,1.000000,0.000000
max,1.000000,1.000000


In [8]:
data['Length']=0
for i in range(len(data.Message)):
    data.loc[i,'Length']=len(data.loc[i,'Message'])

In [9]:
data.head(2)

,Message,label_ham,label_spam,Length
0,"Go until jurong point, crazy.. Available only ...",1,0,111
1,Ok lar... Joking wif u oni...,1,0,29


In [10]:
data['Message'][1]

'Ok lar... Joking wif u oni...'

Preparing the word vetor

In [11]:
corpus=[]

Using Porter Stemmer

In [12]:
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
ps=PorterStemmer()

In [14]:

for i in range(len(data.Message)):
    #regular expressions
    msg=data['Message'][i]
    #email
    msg=re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b','emailaddr',msg)
    # url
    msg=re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",'httpaddr',msg)
    #money
    msg=re.sub('£|\$','moneysymb',msg)
    #phone_numbers
    msg=re.sub(re.compile(r"^\+?[1-9]\d{1,14}$"),'phonenumber',msg)
    #numbers
    msg=re.sub('\d+(\.\d+)?','number',msg)
    #remoing punctuation marks
    msg=re.sub('[^\w\d\s]',' ',msg)

    if(i<=2):
        print('\t\t\t\t\t\t','Message{}'.format(i))
        print("Regular expression:",msg)
    #lowering the case
    msg=msg.lower()

    if(i<=2):
        print("After lowering case:",msg)

    #split
    msg=msg.split()


    if(i<=2):
        print("After splitting:",msg)

    #removal of stop words and stemming
    msg=[ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]

    if(i<=2):
        print("After stemming:",msg)

    #join
    msg=' '.join(msg)

    if(i<=2):
        print("After joining:",msg)

    corpus.append(msg)
    


						 Message0
Regular expression: Go until jurong point  crazy   Available only in bugis n great world la e buffet    Cine there got amore wat   
After lowering case: go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat   
After splitting: ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']
After stemming: ['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat']
After joining: go jurong point crazi avail bugi n great world la e buffet cine got amor wat
						 Message1
Regular expression: Ok lar    Joking wif u oni   
After lowering case: ok lar    joking wif u oni   
After splitting: ['ok', 'lar', 'joking', 'wif', 'u', 'oni']
After stemming: ['ok', 'lar', 'joke', 'wif', 'u', 'oni']
After joining: ok lar joke wif u oni
						 Message2
Regular expressio

CountVectorizer

In [15]:
cv=CountVectorizer()

In [16]:
data_input=cv.fit_transform(corpus).toarray()
data_input[2]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
data_output=data['label_ham']

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data_input,data_output,test_size=0.2,random_state=1)

In [19]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import  accuracy_score, classification_report

Training

In [20]:
model_nb=GaussianNB()
model_nb.fit(x_train,y_train)

model_dt=DecisionTreeClassifier(criterion='entropy',max_depth=3)
model_dt.fit(x_train,y_train)

model_rf=RandomForestClassifier(n_estimators=450,max_depth=3)
model_rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=3, n_estimators=450)

Prediction

In [21]:
#NaiveBayes
y_pred_nb=model_nb.predict(x_test)
print(accuracy_score(y_test,y_pred_nb))
print(classification_report(y_test,y_pred_nb))

0.8798206278026905
              precision    recall  f1-score   support

           0       0.52      0.90      0.66       144
           1       0.98      0.88      0.93       971

    accuracy                           0.88      1115
   macro avg       0.75      0.89      0.79      1115
weighted avg       0.92      0.88      0.89      1115



In [22]:
#DecisionTree
y_pred_dt=model_dt.predict(x_test)
print(accuracy_score(y_test,y_pred_dt))
print(classification_report(y_test,y_pred_dt))

0.9497757847533632
              precision    recall  f1-score   support

           0       0.89      0.70      0.78       144
           1       0.96      0.99      0.97       971

    accuracy                           0.95      1115
   macro avg       0.92      0.84      0.88      1115
weighted avg       0.95      0.95      0.95      1115



In [23]:
#RandomForest
y_pred_rf=model_rf.predict(x_test)
print(accuracy_score(y_test,y_pred_rf))
print(classification_report(y_test,y_pred_rf))

0.8708520179372198
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       144
           1       0.87      1.00      0.93       971

    accuracy                           0.87      1115
   macro avg       0.44      0.50      0.47      1115
weighted avg       0.76      0.87      0.81      1115



C:\Users\ARIJIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ARIJIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ARIJIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: U

Save the model(rf)

In [24]:
import pickle

with open('ham1spam0.pkl','wb') as file:
    pickle.dump(model_nb,file)
with open("vectorizer.pkl","wb") as file:
    pickle.dump(cv,file)

In [25]:
with open('ham1spam0.pkl','rb') as file:
    model=pickle.load(file)


after preprocessing convert into vector

In [26]:

def preprocessAndConvert(msg):
    
    #regular expressions
    #email
    msg=re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b','emailaddr',msg)
    # url
    msg=re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",'httpaddr',msg)
    #money
    msg=re.sub('£|\$','moneysymb',msg)
    #phone_numbers
    msg=re.sub(re.compile(r"^\+?[1-9]\d{1,14}$"),'phonenumber',msg)
    #numbers
    msg=re.sub('\d+(\.\d+)?','number',msg)
    #remoing punctuation marks
    msg=re.sub('[^\w\d\s]',' ',msg)

    #lowering the case
    msg=msg.lower()

    #split
    msg=msg.split()

    #removal of stop words and stemming
    msg=[ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]

    #join
    msg=' '.join(msg)
    
    #convert to 
    with open('vectorizer.pkl','rb') as file:
        vectorizer=pickle.load(file)
        
    vector=vectorizer.transform([msg]).toarray()
    
    return vector


In [27]:
# vt=preprocessAndConvert("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.")
vt=preprocessAndConvert("IMPORTANT: Your account has been flagged for suspicious transactions. To secure your funds, please verify your information here: [scam-link]. Failure to act may result in account suspension.")
pred=model.predict(vt)
print(pred)

[0]
